In [1]:
from google.cloud import bigquery
import pandas as pd
from annotationHelperLib import *
from reportMarkingFunctions import *
from IPython.display import clear_output
import json

In [2]:
# Replace with the name you want used in publications of this data set
name = "Jenna Schabdach"

# Projects: SLIP, SLIP Elementary, SLIP Adolescents, Clinical Imaging Genetics
project = "SLIP Elementary" # Clinical Fetal"

# Replace with the path to your .json file containing a dictionary of all the phrases you'd like to be highlighted
phrasesToHighlightFn = "phrases_to_highlight.json"

# Load the dictionary of phrases to highlight in certain colors 
with open(phrasesToHighlightFn, 'r', encoding='utf-8') as f:
    toHighlight = json.load(f)
    
get_grader_status_report(name)


select * from lab.test_grader_table_with_metadata where grader_name = 'Jenna Schabdach';
151
151
Jenna Schabdach has graded 151 of 151 reliability reports
Jenna Schabdach has graded 37975 unique reports of 37978 assigned where
34163 have been given a grade of 0
1743 have been given a grade of 1
2062 have been given a grade of 2


In [3]:
isOnMainTable = welcome_user(name)

Welcome, Jenna Schabdach
151
151
Jenna Schabdach has graded 151 of 151 reliability reports
You currently have 3 ungraded reports to work on.


In [ ]:
batchSize = 10

for i in range(batchSize):
    clear_output()
    print(i)
    if isOnMainTable:
        mark_one_report_sql(name, project, toHighlight)
    else:
        mark_selfeval_report_sql(name, toHighlight)


2
SELECT * FROM lab.test_grader_table_with_metadata reports join lab.proc_ord_projects projects on (reports.proc_ord_id = projects.proc_ord_id and reports.pat_id = projects.pat_id)  WHERE grader_name = "Jenna Schabdach" and grade = 999 and grade_category = "Unique" LIMIT 1
Year of scan: 2022
Age at scan: 3.43 years
BRAIN MRI, WITHOUT AND WITH CONTRAST: 

CLINICAL INDICATION: Altered mental status and hypoglycemia and hallucinations 

TECHNIQUE: Sagittal 3D T1 gradient echo with axial reformations, axial and coronal TSE T2, axial FLAIR, axial spin echo T1, arterial spin labeled perfusion imaging, axial susceptibility weighted imaging, post-contrast sagittal 3D T1 gradient echo with axial and coronal reformations, post-contrast axial spin echo T1 with fat suppression, post contrast axial flair with fat saturation, axial diffusion imaging were performed on a 3.0 Tesla system. 

COMPARISON: CT brain February 1, 2022 

FINDINGS: THE VENTRICULAR SYSTEM IS NORMAL IN SIZE, SHAPE, AND CONFIGURA

In [5]:
## If/when you run out of reports, you can uncomment the following line of code
#   and run this cell to get 50 more reports from the database. Make sure your
#   values for `project` and `queryFn` match the project you're working on.
# Projects: SLIP Adolescents, SLIP, SLIP Elementary, Clinical Imaging Genetics
# project = "SLIP Adolescents"
# get_second_look_reports_to_grade(name, 5)
get_more_reports_to_grade(name, project, num_to_add=5)

It is expected for this function to take several minutes to run. Your patience is appreciated.
Project: SLIP Elementary
Total reports: 15009 (note each report must be graded by 2 graders)
Graded 0: 805
Graded 1: 2102
Graded 2: 911
Queued: 289
Skipped: 13
insert into lab.test_grader_table_with_metadata (proc_ord_id, grader_name, grade, grade_category, pat_id, age_in_days, proc_ord_year, proc_name, report_origin_table, grade_date) 
        select
          distinct 
          proc_ord.proc_ord_id, "Jenna Schabdach" as grader_name,
          999 as grade,
          "Unique" as grade_category,
          proc_ord.pat_id,
          proc_ord.proc_ord_age as age_in_days,
          proc_ord.proc_ord_year,
          proc_ord.proc_ord_desc as proc_name,
          "arcus.procedure_order" as report_origin_table, "0000-00-00" as grade_date 
        from
          arcus.procedure_order proc_ord
          join arcus.patient pat on proc_ord.pat_id = pat.pat_id
        where
          proc_ord.proc_ord_